In [1]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

def progress_bar(current, total, bar_length=20):
    fraction = current / total

    arrow = int(fraction * bar_length - 1) * '-' + '>'
    padding = int(bar_length - len(arrow)) * ' '

    ending = '\n' if current == total else '\r'

    print(f'Inference progress: [{arrow}{padding}] {int(fraction*100)}%', end=ending)

!pip install -r requirements.txt

fatal: destination path 'pytorch-CycleGAN-and-pix2pix' already exists and is not an empty directory.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [13]:
# direction = "A" # from A to B (purple to brown - 2142 test img)
direction = "B" # from B to A (brown to purple - 469 test img)
model = "livergan_default"

In [14]:
import torch
import torchvision.transforms as transforms
from models.networks import define_G
from collections import OrderedDict
import matplotlib.pyplot as plt

In [15]:
model_dict = torch.load(f"/storage/plzen1/home/javorek/checkpoints/{model}/latest_net_G_{direction}.pth")
new_dict = OrderedDict()
for k, v in model_dict.items():
    # load_state_dict expects keys with prefix 'module.'
    new_dict["module." + k] = v

# make sure you pass the correct parameters to the define_G method
generator_model = define_G(input_nc=3,output_nc=3,ngf=64,netG="resnet_9blocks",
                            norm="instance",use_dropout=False,init_gain=0.02,gpu_ids=[0])
generator_model.load_state_dict(new_dict)

initialize network with normal


<All keys matched successfully>

In [16]:
foldername = f"/storage/plzen1/home/javorek/livergan/test{direction}/"
picnum = len(os.listdir(foldername))
c = 0

from datetime import datetime
a = datetime.now()
print(f"Inference of the model {model} (starting domain = {direction}) started at ", a.strftime("%d/%m/%Y %H:%M:%S"))

for filename in os.listdir(foldername):
    if filename.endswith(".png"):
        file_path = os.path.join(foldername, filename)

        imgA = plt.imread(file_path)
        if imgA.shape[2] == 4:
            imgA = imgA[:, :, :3]

        with torch.no_grad():
            generator_model.eval()

            to_tensor = transforms.ToTensor()
            imgA = to_tensor(imgA)
            imgB = generator_model(imgA)
        imgB = imgB.cpu().permute(1, 2, 0).numpy()
        imgB = (imgB - imgB.min()) / (imgB.max() - imgB.min()) # normalize
        if direction == "A":
            plt.imsave(f'/storage/plzen1/home/javorek/inference/{model}/AtoB/from{direction}_{filename}', imgB)
        elif direction == "B":
            plt.imsave(f'/storage/plzen1/home/javorek/inference/{model}/BtoA/from{direction}_{filename}', imgB)
        progress_bar(c+1, picnum)
        c = c + 1
        
b = datetime.now()
delta = b - a
print(f"Inference of the model {model} (starting domain {direction}) ended at ", b.strftime("%d/%m/%Y %H:%M:%S"))
print(f"Inference of {picnum} images took {delta.total_seconds()} seconds ({picnum/delta.total_seconds()} FPS)")

Inference of the model livergan_full_unet (starting domain = A) started at  08/03/2024 17:13:42


/var/tmp/pbs.20550599.meta-pbs.metacentrum.cz/ipykernel_600634/3271919550.py:24: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  imgA = imgA.resize((768, new_height), Image.ANTIALIAS)


RuntimeError: Given transposed=1, weight of size [1024, 512, 4, 4], expected input[1, 512, 12, 6] to have 1024 channels, but got 512 channels instead